## Import Libraries and Stuff

In [ ]:
import numpy as np
import torch
from torch import optim
import random

In [ ]:
#Code for ASGD ported from https://github.com/rahulkidambi/AccSGD

from torch.optim.optimizer import Optimizer, required
import copy

class AccSGD(Optimizer):
    r"""Implements the algorithm proposed in https://arxiv.org/pdf/1704.08227.pdf, which is a provably accelerated method 
    for stochastic optimization. This has been employed in https://openreview.net/forum?id=rJTutzbA- for training several 
    deep learning models of practical interest. This code has been implemented by building on the construction of the SGD 
    optimization module found in pytorch codebase.
    Args:
        params (iterable): iterable of parameters to optimize or dicts defining
            parameter groups
        lr (float): learning rate (required)
        kappa (float, optional): ratio of long to short step (default: 1000)
        xi (float, optional): statistical advantage parameter (default: 10)
        smallConst (float, optional): any value <=1 (default: 0.7)
    Example:
        >>> from AccSGD import *
        >>> optimizer = AccSGD(model.parameters(), lr=0.1, kappa = 1000.0, xi = 10.0)
        >>> optimizer.zero_grad()
        >>> loss_fn(model(input), target).backward()
        >>> optimizer.step()
    """

    def __init__(self, params, lr=0.001, kappa = 1000.0, xi = 10.0, smallConst = 0.7, weight_decay=0):
        defaults = dict(lr=lr, kappa=kappa, xi=xi, smallConst=smallConst,
                        weight_decay=weight_decay)
        super(AccSGD, self).__init__(params, defaults)

    def __setstate__(self, state):
        super(AccSGD, self).__setstate__(state)

    def step(self, closure=None):
        """ Performs a single optimization step.
        Arguments:
            closure (callable, optional): A closure that reevaluates the model
                and returns the loss.
        """
        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:
            weight_decay = group['weight_decay']
            large_lr = (group['lr']*group['kappa'])/(group['smallConst'])
            Alpha = 1.0 - ((group['smallConst']*group['smallConst']*group['xi'])/group['kappa'])
            Beta = 1.0 - Alpha
            zeta = group['smallConst']/(group['smallConst']+Beta)
            for p in group['params']:
                if p.grad is None:
                    continue
                d_p = p.grad.data
                if weight_decay != 0:
                    d_p.add_(weight_decay, p.data)
                param_state = self.state[p]
                if 'momentum_buffer' not in param_state:
                    param_state['momentum_buffer'] = copy.deepcopy(p.data)
                buf = param_state['momentum_buffer']
                buf.mul_((1.0/Beta)-1.0)
                buf.add_(-large_lr,d_p)
                buf.add_(p.data)
                buf.mul_(Beta)

                p.data.add_(-group['lr'],d_p)
                p.data.mul_(zeta)
                p.data.add_(1.0-zeta,buf)

        return loss



In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

#Randomly select a weight matrix such as
# w = torch.rand(2,1)
# Ensure that you're using the same 'w' used in LinearRegression.ipynb for grid search to make sense
w = torch.tensor([[0.5],[2]])

#Define Gaussian Distribution as per Original Paper
def gaussian(N,k):    
    a = torch.distributions.multivariate_normal.MultivariateNormal(
        loc=torch.Tensor([0,0]), covariance_matrix=torch.Tensor([[1, 0],[0, 1/k]])).sample((N,))
    b = a@w
    if torch.cuda.is_available(): 
        return Variable(a).cuda(),Variable(b).cuda()
    else:
        return Variable(a),Variable(b)

#Define Discrete Distribution as per Original Paper
def discrete(N,k):
  a = torch.zeros(N,2)
  b = torch.zeros(N,1)

  for i in range(N):
    dist = torch.multinomial(input= torch.tensor([0.5, 0.5]),num_samples=2).float()
    a[i] = dist

  a[:,1]=a[:,1].float()*(2.0/k)
  b = a @ w  

  if torch.cuda.is_available(): 
        return Variable(a).cuda(),Variable(b).cuda()
  else:
        return Variable(a),Variable(b)

#Define a simplistic Neural Network for Linear Regression
class LinearRegression(nn.Module):
    def __init__(self):
        super(LinearRegression,self).__init__()
        self.linear = nn.Linear(2,1)

    def forward(self,input):
        output = self.linear(input)
        return output

## 1. SGD Grid Search

In [ ]:
def error_SGD(lr, n = 9, Gaussian=False):

  seed = 10
  torch.manual_seed(seed)
  torch.backends.cudnn.deterministic = True
  
  Num_samples = 1000
  k_set = torch.zeros(n)

  #Maintain loss list having loss values for each Condition Number k, i.e., a global loss list
  error_list = []
    
  #Iterate over each possible value of Condition Number k
  for i in range (n):
    k = 2**(i+4)
    k_set[i] = k

    #Maintain a per Condition Number Loss List, i.e., a local loss list
    per_k_error_list = []

    iterations = 5*k

    loss_list = torch.zeros(9, iterations).cuda()

    #Generate Training Data
    if Gaussian:
      X, y = gaussian(Num_samples, k_set[i])
    else:
      X, y = discrete(Num_samples, k_set[i])

    model = LinearRegression().cuda()
    optimizer = optim.SGD(model.parameters(), lr=lr)
    loss_func = nn.MSELoss()

    initial_loss = 0
    count = 0


    for t in range(iterations):

        y_hat = model(X)
        loss = loss_func(y_hat,y)
        loss_list[i][t] = loss.item()
        
        if t == 0:
          initial_loss = loss.item()
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()

        #Verify Geometric Convergence Criteria
        elif t > int(iterations/2):
            
            #If convergence criteria has not been violated, append the loss value to the local loss list
            if loss.item() < initial_loss:
              r =  loss.item()
              per_k_error_list.append(r)
              optimizer.zero_grad()
              loss.backward()
              optimizer.step()
    
            #If convergence criteria fails at any one loss value (and therefore t),
            #straight away assign infinite loss to the particular hyperparameter setting
            else:
              print(loss.item(), initial_loss)
              print("Violation of Convergence criteria at learning rate: ",lr," k: ",k, " epoch: ",t, "iter: ",iterations)
              print("Assigning average infinite loss to learning rate: ", lr)
              per_k_error_list = []
              return float('inf')

        else:
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()

        count += 1

    
    print("------At k = {}, Size of Sub-Converge List of Loss Values: {}".format(k, len(per_k_error_list))) 

    #Randomly sample at max 100 loss values from the local loss list and append them to the global loss list
    if len(per_k_error_list) > 100:
      per_k_error_list = random.sample(per_k_error_list, k=100)

    for error in per_k_error_list:
      error_list.append(error)

  print("--Size of Converge List across all values of k on Random Sampling from Each Sub-Converge List: ",len(error_list))

  #Randomly sample at max 100 loss values from the global loss list
  if len(error_list) > 100:
      error_list = random.sample(error_list, k=100)

  print("--Size of Converge List on re-random sampling (at max 100 values): ",len(error_list))

  #If loss list is empty, return infinite loss. This corresponds to violation of Convergence Criteria somewhere before
  if len(error_list) == 0:
    return float('inf')
  
  #Return the average of all the sampled loss values as final loss at a particular hyperparameter setting
  return sum(error_list)/len(error_list)

In [ ]:
#Create Learning Rate Set as {0.01, 0.02, ...., 0.99, 1} for Grid Search
learning_rate_set = np.linspace(0.01,1,99)
#Create Loss Set to store the loss values for each learning rate above
loss_set_SGD = np.zeros(99)

min_error = 0
x = 0

#Iterate over Learning Rate Set
for lr in learning_rate_set:

    #Compute the loss for a particular hyperparameter setting
    error = error_SGD(lr, Gaussian=True)
    loss_set_SGD[x] = error

    #Update the minimum loss value and the corresponding optmal hyperparameter value found so far
    if (min_error == 0 or error<min_error) and error!=0:
      min_lr = lr
      min_x = x
      min_error = error
    print("Learning Rate: {}  has Average Loss Value: {}\n".format(lr, error))

    x += 1

In [ ]:
#Print the entire loss set as well the optimal settings found using Grid Search
print("Loss Set for SGD :")
print(loss_set_SGD)
print("Minimum Loss found is: {} @ Learning Rate {}".format(loss_set_SGD[min_x], min_lr))

## 2. Heavy Ball Grid Search

In [ ]:
def error_HB(lr, momentum, n = 9, Gaussian=False):

  seed = 7
  torch.manual_seed(seed)
  torch.backends.cudnn.deterministic = True

  Num_samples = 1000
  k_set = torch.zeros(n)

  error_list = []

  #Iterate over each possible value of Condition Number k
  for i in range (n):
    k = 2**(i+4)
    k_set[i] = k

    #Maintain a per Condition Number Loss List, i.e., a local loss list
    per_k_error_list = []

    iterations = 5*k

    loss_list = torch.zeros(9, iterations).cuda()

    #Generate Training Data
    if Gaussian:
      X, y = gaussian(Num_samples, k_set[i])
    else:
      X, y = discrete(Num_samples, k_set[i])

    model = LinearRegression().cuda()
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum = momentum)
    loss_func = nn.MSELoss()

    initial_loss = 0
    count = 0


    for t in range(iterations):

        y_hat = model(X)
        loss = loss_func(y_hat,y)
        loss_list[i][t] = loss.item()

        if t == 0:
          initial_loss = loss.item()
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()

        #Verify Geometric Convergence Criteria
        elif t > int(iterations/2):

            #If convergence criteria has not been violated, append the loss value to the local loss list
            if loss.data < initial_loss:
              r =  loss.item()
              per_k_error_list.append(r)
              optimizer.zero_grad()
              loss.backward()
              optimizer.step()

            #If convergence criteria fails at any one loss value (and therefore t),
            #straight away assign infinite loss to the particular hyperparameter setting 
            else:
              print(loss.item(), initial_loss)
              print("Violation of Convergence criteria at learning rate: ",lr," momentum: ",momentum," k: ",k, " epoch: ",t, "iter: ",iterations)
              print("Assigning average infinite loss to learning rate: ", lr)
              per_k_error_list = []
              return float('inf')

        else:
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()

        count += 1

    print("------At k = {}, Size of Sub-Converge List of Loss Values: {}".format(k, len(per_k_error_list))) 

    #Randomly sample at max 100 loss values from the local loss list and append them to the global loss list
    if len(per_k_error_list) > 100:
      per_k_error_list = random.sample(per_k_error_list, k=100)

    for error in per_k_error_list:
      error_list.append(error)

  print("--Size of Converge List across all values of k on Random Sampling from Each Sub-Converge List: ",len(error_list))

  #Randomly sample at max 100 loss values from the global loss list
  if len(error_list) > 100:
      error_list = random.sample(error_list, k=100)

  print("--Size of Converge List on re-random sampling (at max 100 values): ",len(error_list))

  #If loss list is empty, return infinite loss. This corresponds to violation of Convergence Criteria somewhere before
  if len(error_list) == 0:
    return float('inf')

  #Return the average of all the sampled loss values as final loss at a particular hyperparameter setting
  return sum(error_list)/len(error_list)

In [ ]:
#Create Learning Rate Set as {0.1, 0.2, ...., 0.9, 1} for Grid Search
learning_rate_set = np.linspace(0.1,1,10)
#Create Momentum Set as {0.1, 0.2, ...., 0.9, 1} for Grid Search
momentum_set = np.linspace(0.1,1,10)
loss_set_HB = np.zeros([10,10])

min_error = 0
x = 0

#Iterate over learning rate set
for lr in learning_rate_set:
    y = 0

    #Iterate over Momentum set
    for m in momentum_set:

        #Compute the loss for a particular hyperparameter setting
        error = error_HB(lr, m, Gaussian=True)
        loss_set_HB[x,y] = error
    
        #Update the minimum loss value and the corresponding optmal hyperparameter value found so far
        if (min_error == 0 or error<min_error) and error!=0:
          min_lr = lr
          min_m = m
          min_x = x
          min_y = y
          min_error = error

        print("Learning Rate: {}, Momentum: {}  has Average Loss Value: {}\n".format(lr, m, error))
        y += 1

    x += 1

In [ ]:
#Print the entire loss set as well the optimal settings found using Grid Search
print("Loss Set for Heavy Ball :")
print(loss_set_HB)
print("Minimum Loss found is: {} @ Learning Rate {}, Momentum: {}".format(loss_set_HB[min_x], min_lr, min_m))

## 3. NAG Grid Search

In [ ]:
def error_NAG(lr, momentum, n = 9):

  seed = 7
  torch.manual_seed(seed)
  torch.backends.cudnn.deterministic = True

  Num_samples = 1000
  k_set = torch.zeros(n)

  error_list = []

  #Iterate over each possible value of Condition Number k 
  for i in range (n):
    k = 2**(i+4)
    k_set[i] = k

    #Maintain a per Condition Number Loss List, i.e., a local loss list
    per_k_error_list = []

    iterations = 5*k

    loss_list = torch.zeros(9, iterations).cuda()

    #Generate Training Data
    if Gaussian:
      X, y = gaussian(Num_samples, k_set[i])
    else:
      X, y = discrete(Num_samples, k_set[i])

    model = LinearRegression().cuda()
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum = momentum, nesterov = True)
    loss_func = nn.MSELoss()

    initial_loss = 0
    count = 0


    for t in range(iterations):

        y_hat = model(X)
        loss = loss_func(y_hat,y)
        loss_list[i][t] = loss.item()

        if loss.item() == 0:
            return float('inf')

        if t == 0:
          initial_loss = loss.item()
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()

        #Verify Geometric Convergence Criteria
        elif t > int(iterations/2):

            #If convergence criteria has not been violated, append the loss value to the local loss list
            if loss.item() < initial_loss:
              r =  loss.item()
              per_k_error_list.append(r)
              optimizer.zero_grad()
              loss.backward()
              optimizer.step()

            #If convergence criteria fails at any one loss value (and therefore t),
            #straight away assign infinite loss to the particular hyperparameter setting    
            else:
              print(loss.item(), initial_loss)
              print("Violation of Convergence criteria at learning rate: ",lr," momentum: ",momentum," k: ",k, " epoch: ",t, "iter: ",iterations)
              print("Assigning average infinite loss to learning rate: ", lr)
              per_k_error_list = []
              return float('inf')

        else:
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()

        count += 1

    print("------At k = {}, Size of Sub-Converge List of Loss Values: {}".format(k, len(per_k_error_list))) 

    #Randomly sample at max 100 loss values from the local loss list and append them to the global loss list
    if len(per_k_error_list) > 100:
      per_k_error_list = random.sample(per_k_error_list, k=100)

    for error in per_k_error_list:
      error_list.append(error)

  print("--Size of Converge List across all values of k on Random Sampling from Each Sub-Converge List: ",len(error_list))

  #Randomly sample at max 100 loss values from the global loss list
  if len(error_list) > 100:
      error_list = random.sample(error_list, k=100)

  print("--Size of Converge List on re-random sampling (at max 100 values): ",len(error_list))

  #If loss list is empty, return infinite loss. This corresponds to violation of Convergence Criteria somewhere before
  if len(error_list) == 0:
    return float('inf')

  #Return the average of all the sampled loss values as final loss at a particular hyperparameter setting
  return sum(error_list)/len(error_list)

In [ ]:
learning_rate_set = np.linspace(0.1,1,10)
momentum_set = np.linspace(0.1,1,10)
loss_set_NAG = np.zeros([10,10])

min_error = 1e8
x = 0
for lr in learning_rate_set:
    y = 0
    for m in momentum_set:

        #Compute the loss for a particular hyperparameter setting
        error = error_NAG(lr, m, Gaussian=True)
        loss_set_NAG[x,y] = error

        #Update the minimum loss value and the corresponding optmal hyperparameter value found so far
        if (min_error == 1e8 or error<min_error) and error!=0:
          min_lr = lr
          min_m = m
          min_x = x
          min_y = y
          min_error = error

        print("Learning Rate: {}, Momentum: {}  has Average Loss Value: {}\n".format(lr, m, error))
        y += 1

    x += 1

In [ ]:
#Print the entire loss set as well the optimal settings found using Grid Search
print("Loss Set for Heavy Ball :")
print(loss_set_NAG)
print("Minimum Loss found is: {} @ Learning Rate {}, Momentum: {}".format(loss_set_NAG[min_x], min_lr, min_m))

## ASGD Grid Search

In [ ]:
def error_ASGD(lr, n = 9, Gaussian=False):

  seed = 10
  torch.manual_seed(seed)
  torch.backends.cudnn.deterministic = True

  Num_samples = 1000
  k_set = torch.zeros(n)

  error_list = []

  #Iterate over each possible value of Condition Number k
  for i in range (n):
    k = 2**(i+4)
    k_set[i] = k

    per_k_error_list = []

    iterations = 5*k

    llr = 2*k
    sap = np.sqrt(2/3 * k)

    loss_list = torch.zeros(9, iterations).cuda()

    #Generate Training Data
    if Gaussian:
      X, y = gaussian(Num_samples, k_set[i])
    else:
      X, y = discrete(Num_samples, k_set[i])

    model = LinearRegression().cuda()
    optimizer = AccSGD(model.parameters(), lr=lr,  kappa = llr, xi = sap)    
    loss_func = nn.MSELoss()

    initial_loss = 0
    count = 0


    for t in range(iterations):

        y_hat = model(X)
        loss = loss_func(y_hat,y)
        loss_list[i][t] = loss.item()

        if t == 0:
          initial_loss = loss.item()
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()

        #Verify Geometric Convergence Criteria
        elif t > int(iterations/2):

            #If convergence criteria has not been violated, append the loss value to the local loss list
            if loss.item() < initial_loss:
              r =  loss.item()
              per_k_error_list.append(r)
              optimizer.zero_grad()
              loss.backward()
              optimizer.step()

            #If convergence criteria fails at any one loss value (and therefore t),
            #straight away assign infinite loss to the particular hyperparameter setting  
            else:
              print(loss.item(), initial_loss)
              print("Violation of Convergence criteria at learning rate: ",lr," k: ",k, " epoch: ",t, "iter: ",iterations)
              print("Assigning average infinite loss to learning rate: ", lr)
              per_k_error_list = []
              return float('inf')

        else:
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()

        count += 1

    print("------At k = {}, Size of Sub-Converge List of Loss Values: {}".format(k, len(per_k_error_list))) 

    #Randomly sample at max 100 loss values from the local loss list and append them to the global loss list
    if len(per_k_error_list) > 100:
      per_k_error_list = random.sample(per_k_error_list, k=100)

    for error in per_k_error_list:
      error_list.append(error)

  print("--Size of Converge List across all values of k on Random Sampling from Each Sub-Converge List: ",len(error_list))

  #Randomly sample at max 100 loss values from the global loss list
  if len(error_list) > 100:
      error_list = random.sample(error_list, k=100)

  print("--Size of Converge List on re-random sampling (at max 100 values): ",len(error_list))

  #If loss list is empty, return infinite loss. This corresponds to violation of Convergence Criteria somewhere before
  if len(error_list) == 0:
    return float('inf')

  #Return the average of all the sampled loss values as final loss at a particular hyperparameter setting
  return sum(error_list)/len(error_list)

In [ ]:
#Create Learning Rate Set as {0.01, 0.02, ...., 0.99, 1} for Grid Search
learning_rate_set = np.linspace(0.01,1,99)
#Create Loss Set to store the loss values for each learning rate above
loss_set_ASGD = np.zeros([99])

min_error = 0
x = 0

#Iterate over Learning Rate Set
for lr in learning_rate_set:

    #Compute the loss for a particular hyperparameter setting
    error = error_ASGD(lr, Gaussian=True)
    loss_set_ASGD[x] = error

    #Update the minimum loss value and the corresponding optmal hyperparameter value found so far
    if (min_error == 0 or error<min_error) and error!=0:
      min_lr = lr
      min_x = x
      min_error = error
    print("Learning Rate: {}  has Average Loss Value: {}\n".format(lr, error))

    x += 1

In [ ]:
#Print the entire loss set as well the optimal settings found using Grid Search
print("Loss Set for ASGD :")
print(loss_set_ASGD)
print("Minimum Loss found is: {} @ Learning Rate {}".format(loss_set_ASGD[min_x], min_lr))